# Recommender System
## Group 7

This shows some attempts to build a recommender system

In [1]:
# import libraries
import graphlab as gl
import numpy as np
import pandas as pd

## Input data : keep a dataframe containing user-item interactions

In [2]:
# create SFrame for London twitter data
london_sf = gl.SFrame.read_csv('../DataSources/FourCityDataset/twitter/london.txt',
                        delimiter=' ',header=False)
london_sf.rename({'X1':'user_id','X2':'venue_id','X3':'#visits(normalized)'})

This non-commercial license of GraphLab Create for academic use is assigned to Y.Li-25@student.tudelft.nl and will expire on February 22, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\xps\AppData\Local\Temp\graphlab_server_1491227855.log.0


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\london.txt

Parsing completed. Parsed 100 lines in 0.22273 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\london.txt

Parsing completed. Parsed 48852 lines in 0.184915 secs.

user_id,venue_id,#visits(normalized)
8493742,4ac518c5f964a520e1a420e3,0.666666666667
28777828,4c2ceb692219c92835e5a648,0.666666666667
162511952,4babef9df964a52012d73ae3,0.75
18816105,4aeb31a3f964a5207bbf21e3,0.666666666667
18319254,4af7074ff964a520fa0422e3,0.666666666667
376938788,4b7c1c76f964a520917d2fe3,0.8
20669549,4ac518e7f964a52053ab20e3,0.666666666667
20735661,4b838e75f964a520a30931e3,0.75
138078717,4f5a594fe4b0b04917548235,0.666666666667
139888796,4c177dffd4ee9521ba37ba05,0.666666666667


## Clean interactions, remove places contain "home"

In [3]:
london_sf = london_sf[london_sf['venue_id'].apply(lambda x: x.find('home'))==-1]

In [4]:
len(london_sf['venue_id'].unique())

11976

## Check unique users and items

In [117]:
print 'Unique users:',len(london_sf['user_id'].unique()) 
print 'Unique venues:',len(london_sf['venue_id'].unique())
print 'Total interactions:', len(london_sf)

Unique users: 9305
Unique venues: 11976
Total interactions: 46711


In [118]:
#category_list = london_sf_joined['category'].unique()
#delete_list = ['Fire Station','Physical Therapist',
#               'Animal Shelter','Housing Development',"Doctor's Office",
#              'Optical','Recruiting Agency','Real Estate',"Dentist's Office",
#             'Medical','Lawyer','Residential','Courthouse','Mental Health',
#              'Police Station','Prison','Administrative Building','Parking',
#             'Bus Station','Bus Stop','Bus','Office','Medical Lab','Emergency Room']

In [119]:
#london_sf_joined = london_sf_joined[london_sf_joined['category']
#                                    .apply(lambda x: x not in delete_list)]

## Add interaction data from Instagram user
still in question. Should we add it or not?

## Keep another list for item metadata
todo: enrich the metadata from foursquare

In [5]:
#items = gl.SFrame.read_csv(...)
import json
file_path = './cleaned_london_venues_data_twitter.json'
json_file = open(file_path)
jsonObj = json.load(json_file)
df = pd.DataFrame.from_dict(jsonObj,orient='index')
df['venue_id'] = df.index
sf = gl.SFrame(df)
sf_cleaned = sf['venue_id','name','category','rating','likes','checkins','tags','photo']
item_data = sf_cleaned
item_data = item_data['venue_id','category','rating','likes','checkins']
item_data_temp =item_data.dropna()
mean = item_data_temp['rating'].mean()
item_data=item_data.fillna('rating',mean)
item_data=item_data.fillna('likes',0)
item_data=item_data.fillna('checkins',0)
item_data.head()

venue_id,category,rating,likes,checkins
4a5f9446f964a520e0bf1fe3,Burgers,8.3,150.0,2419
4abcec53f964a520b98720e3,Hotel,9.0,32.0,1510
4abe4502f964a520558c20e3,Palace,9.0,5469.0,92542
4ac3ba25f964a520919c20e3,Hotel,7.2,4.0,303
4ac51183f964a52046a020e3,Portuguese,8.2,31.0,365
4ac51183f964a52048a020e3,Zoo,8.9,823.0,14252
4ac51183f964a52049a020e3,Art Gallery,9.5,1281.0,17161
4ac518b4f964a52067a020e3,Hotel,8.3,60.0,1893
4ac518b4f964a52071a020e3,Hotel,9.2,313.0,11020
4ac518b4f964a52073a020e3,Hotel,8.9,30.0,596


In [6]:
#sf_cleaned[sf_cleaned['photo']==[]]
def first_url(url_list):
    if len(url_list)==0:
        return ''
    else:
        return url_list[0]
sf_cleaned['photo'] = sf_cleaned['photo'].apply(lambda x: first_url(x))

In [7]:
london_sf_joined = london_sf.join(sf['venue_id','name','category'],on='venue_id')

# Keep a table for user infomation

In [8]:
user_data = gl.SFrame.read_csv('../DataSources/FourCityDataset/twitter/twitter_profiles.csv')

Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\twitter_profiles.csv

Parsing completed. Parsed 100 lines in 2.15294 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,str,str,str,str,float,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,float,float,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\twitter_profiles.csv

Parsing completed. Parsed 21024 lines in 0.976184 secs.

In [9]:
user_data = user_data['id','g_post_count','g_home_city','g_home_country','g_country_count','g_gender','g_age','l_city','uclass']
user_data = user_data[user_data['l_city']=='london']

In [10]:
user_data.rename({'id':'user_id'})
user_data=user_data['user_id','g_gender','g_age']

In [11]:
# filter users in our interaction dataset
user_data=user_data[user_data['user_id'].apply(lambda x:x in set(london_sf_joined['user_id']))]

In [13]:
len(user_data)

4248

## Create a simple model and evaluate it
1. Random split data into training data and validation data
2. We have a target column: #visits, thus we can create a model to predict which new places a user would visit frequently.

In [61]:
#train_data, test_data = gl.recommender.util.random_split_by_user(london_sf, 'user_id', 'venue_id')
#train_data,test_data = london_sf_joined.random_split(.8)
folds = gl.cross_validation.KFold(london_sf_joined, 10)
valid_error=[]
for train, valid in folds:
    m = gl.ranking_factorization_recommender.create(train,
                                                    user_id='user_id',
                                                    item_id='venue_id',
                                                    target = "#visits(normalized)",
                                                    user_data=user_data,item_data=item_data
                                                   )
    valid_error += [m.evaluate_rmse(valid,target='#visits(normalized)')['rmse_overall']]
print "average validation RMSE is:%f" %(sum(valid_error)/len(valid_error))

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 42039 observations with 9303 users and 11976 items.

Data prepared in: 0.463123s

Training ranking_factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.000289647

Final training RMSE: 0.008099

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [74]:
print "average validation RMSE is:%f" %(sum(valid_error)/len(valid_error))

average validation RMSE is:0.083015


In [75]:
final_model = gl.ranking_factorization_recommender.create(london_sf_joined,
                                                    user_id='user_id',
                                                    item_id='venue_id',
                                                    target = "#visits(normalized)",
                                                    user_data=user_data,item_data=item_data
                                                   )


Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 46711 observations with 9305 users and 11976 items.

Data prepared in: 0.483464s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Convergence on objective within bounds.

Computing final objective value and training RMSE.

Final objective value: 0.000296426

Final training RMSE: 0.00827918

## Use the model created to recommend places to a user

In [128]:
gl.canvas.set_target('ipynb')
view =final_model.views.overview(user_data=user_data,
                                         item_data=sf_cleaned,
                                         item_description_column='category',
                                         item_name_column='name',
                                         item_url_column='photo')
view.show()

recommendations finished on 1000/5647 queries. users per second: 385.5

recommendations finished on 2000/5647 queries. users per second: 387.158

recommendations finished on 3000/5647 queries. users per second: 380.021

recommendations finished on 4000/5647 queries. users per second: 379.771

recommendations finished on 5000/5647 queries. users per second: 379.002

recommendations finished on 1000/5647 queries. users per second: 11814.9

recommendations finished on 2000/5647 queries. users per second: 9733.4

recommendations finished on 3000/5647 queries. users per second: 9583.01

recommendations finished on 4000/5647 queries. users per second: 9498.25

recommendations finished on 5000/5647 queries. users per second: 9421.38

## create a popularity model as our baseline
This model just recommend based on popularity, no personalization

In [70]:
popularity_model = gl.popularity_recommender.create(train_data,
                                                    user_id='user_id',
                                                    item_id='venue_id',
                                                    target = "#visits(normalized)" )

Recsys training: model = popularity

Warning: Ignoring columns name, category;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 37419 observations with 9246 users and 10885 items.

Data prepared in: 0.069021s

37419 observations to process; with 10885 unique items.

## Make recommendations

In [76]:
recommend_table = final_model.recommend(k=3)

recommendations finished on 1000/9305 queries. users per second: 154.353

recommendations finished on 2000/9305 queries. users per second: 203.396

recommendations finished on 3000/9305 queries. users per second: 242.438

recommendations finished on 4000/9305 queries. users per second: 268.337

recommendations finished on 5000/9305 queries. users per second: 286.575

recommendations finished on 6000/9305 queries. users per second: 299.839

recommendations finished on 7000/9305 queries. users per second: 308.375

recommendations finished on 8000/9305 queries. users per second: 315.333

recommendations finished on 9000/9305 queries. users per second: 315.766

In [79]:
recommend_table

user_id,List of venue_id
225059732,"[4f4b60d5e4b0c132f3e0df85, ..."
20063137,"[522f448a11d29dcb40ba392b, ..."
1181033244,"[4ac518d2f964a52026a720e3, ..."
10356152,"[4be52e42910020a160b5d214, ..."
285211221,"[4be52e42910020a160b5d214, ..."
18157365,"[4ac518cef964a520f8a520e3, ..."
808868670,"[4ac518d5f964a52013a820e3, ..."
236111793,"[4f4b60d5e4b0c132f3e0df85, ..."
21941712,"[4edce907c2ee3cd644c79c65, ..."
332552379,"[4ae6cb07f964a5204ca721e3, ..."


In [ ]:
import graphlab.aggregate as agg
#recommend_table=recommend_table.groupby(key_columns='user_id',operations=agg.CONCAT('venue_id'))
recommend_table.export_json('rec_by_records.json',orient='records')
#recommend_table.export_json('rec_by_rows.json', orient='rows')